# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 20 2022 11:39AM,237738,19,ACG-2102490136,ACG North America LLC,Released
1,May 20 2022 11:23AM,237737,18,HH6070,Hush Hush,Released
2,May 20 2022 10:35AM,237735,10,Enova-9998,Emerginnova,Released
3,May 20 2022 10:22AM,237734,10,8041657,Eywa Pharma Inc.,Released
4,May 20 2022 10:22AM,237734,10,8041691,Eywa Pharma Inc.,Released
5,May 20 2022 10:22AM,237734,10,8041695,Eywa Pharma Inc.,Released
6,May 20 2022 10:22AM,237734,10,8041696,Eywa Pharma Inc.,Released
7,May 20 2022 10:22AM,237734,10,8041699,Eywa Pharma Inc.,Released
8,May 20 2022 10:02AM,237731,20,SSNA-2173612911,Sartorius Stedim North America,Released
9,May 20 2022 9:51AM,237729,10,CTF0011251,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,237731,Released,1
30,237734,Released,5
31,237735,Released,1
32,237737,Released,1
33,237738,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237731,NaN,NaN,1.0
237734,NaN,NaN,5.0
237735,NaN,NaN,1.0
237737,NaN,NaN,1.0
237738,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3.0,0.0,2.0
237650,0.0,0.0,1.0
237654,0.0,1.0,0.0
237655,0.0,0.0,5.0
237659,0.0,0.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3,0,2
237650,0,0,1
237654,0,1,0
237655,0,0,5
237659,0,0,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,0,2
1,237650,0,0,1
2,237654,0,1,0
3,237655,0,0,5
4,237659,0,0,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,,2
1,237650,,,1
2,237654,,1,
3,237655,,,5
4,237659,,,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 20 2022 11:39AM,237738,19,ACG North America LLC
1,May 20 2022 11:23AM,237737,18,Hush Hush
2,May 20 2022 10:35AM,237735,10,Emerginnova
3,May 20 2022 10:22AM,237734,10,Eywa Pharma Inc.
8,May 20 2022 10:02AM,237731,20,Sartorius Stedim North America
9,May 20 2022 9:51AM,237729,10,"Citieffe, Inc."
10,May 20 2022 9:49AM,237728,10,ISDIN Corporation
11,May 20 2022 9:47AM,237726,10,"Methapharm, Inc."
12,May 20 2022 9:40AM,237725,20,Sartorius Stedim North America
13,May 20 2022 9:26AM,237724,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 20 2022 11:39AM,237738,19,ACG North America LLC,,,1
1,May 20 2022 11:23AM,237737,18,Hush Hush,,,1
2,May 20 2022 10:35AM,237735,10,Emerginnova,,,1
3,May 20 2022 10:22AM,237734,10,Eywa Pharma Inc.,,,5
4,May 20 2022 10:02AM,237731,20,Sartorius Stedim North America,,,1
5,May 20 2022 9:51AM,237729,10,"Citieffe, Inc.",,,1
6,May 20 2022 9:49AM,237728,10,ISDIN Corporation,,,1
7,May 20 2022 9:47AM,237726,10,"Methapharm, Inc.",,,1
8,May 20 2022 9:40AM,237725,20,Sartorius Stedim North America,,,1
9,May 20 2022 9:26AM,237724,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 11:39AM,237738,19,ACG North America LLC,1,,
1,May 20 2022 11:23AM,237737,18,Hush Hush,1,,
2,May 20 2022 10:35AM,237735,10,Emerginnova,1,,
3,May 20 2022 10:22AM,237734,10,Eywa Pharma Inc.,5,,
4,May 20 2022 10:02AM,237731,20,Sartorius Stedim North America,1,,
5,May 20 2022 9:51AM,237729,10,"Citieffe, Inc.",1,,
6,May 20 2022 9:49AM,237728,10,ISDIN Corporation,1,,
7,May 20 2022 9:47AM,237726,10,"Methapharm, Inc.",1,,
8,May 20 2022 9:40AM,237725,20,Sartorius Stedim North America,1,,
9,May 20 2022 9:26AM,237724,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 11:39AM,237738,19,ACG North America LLC,1,,
1,May 20 2022 11:23AM,237737,18,Hush Hush,1,,
2,May 20 2022 10:35AM,237735,10,Emerginnova,1,,
3,May 20 2022 10:22AM,237734,10,Eywa Pharma Inc.,5,,
4,May 20 2022 10:02AM,237731,20,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 11:39AM,237738,19,ACG North America LLC,1.0,NaN,NaN
1,May 20 2022 11:23AM,237737,18,Hush Hush,1.0,NaN,NaN
2,May 20 2022 10:35AM,237735,10,Emerginnova,1.0,NaN,NaN
3,May 20 2022 10:22AM,237734,10,Eywa Pharma Inc.,5.0,NaN,NaN
4,May 20 2022 10:02AM,237731,20,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 11:39AM,237738,19,ACG North America LLC,1.0,0.0,0.0
1,May 20 2022 11:23AM,237737,18,Hush Hush,1.0,0.0,0.0
2,May 20 2022 10:35AM,237735,10,Emerginnova,1.0,0.0,0.0
3,May 20 2022 10:22AM,237734,10,Eywa Pharma Inc.,5.0,0.0,0.0
4,May 20 2022 10:02AM,237731,20,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2614819,18.0,15.0,0.0
15,950829,17.0,44.0,2.0
17,237654,0.0,1.0,0.0
18,475437,2.0,0.0,0.0
19,713149,6.0,1.0,0.0
20,1426049,32.0,13.0,3.0
21,237724,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2614819,18.0,15.0,0.0
1,15,950829,17.0,44.0,2.0
2,17,237654,0.0,1.0,0.0
3,18,475437,2.0,0.0,0.0
4,19,713149,6.0,1.0,0.0
5,20,1426049,32.0,13.0,3.0
6,21,237724,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,18.0,15.0,0.0
1,15,17.0,44.0,2.0
2,17,0.0,1.0,0.0
3,18,2.0,0.0,0.0
4,19,6.0,1.0,0.0
5,20,32.0,13.0,3.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,15,Released,17.0
2,17,Released,0.0
3,18,Released,2.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,17,18,19,20,21
Status,,,,,,,
Completed,0.0,2.0,0.0,0.0,0.0,3.0,0.0
Executing,15.0,44.0,1.0,0.0,1.0,13.0,0.0
Released,18.0,17.0,0.0,2.0,6.0,32.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,17,18,19,20,21
0,Completed,0.0,2.0,0.0,0.0,0.0,3.0,0.0
1,Executing,15.0,44.0,1.0,0.0,1.0,13.0,0.0
2,Released,18.0,17.0,0.0,2.0,6.0,32.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,17,18,19,20,21
0,Completed,0.0,2.0,0.0,0.0,0.0,3.0,0.0
1,Executing,15.0,44.0,1.0,0.0,1.0,13.0,0.0
2,Released,18.0,17.0,0.0,2.0,6.0,32.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()